#  Import images

In [1]:
#
# Term project for AMLS_ELEC0134
#
%matplotlib inline
import matplotlib.pyplot as mplp
import matplotlib.image as mpimg
from keras.preprocessing import image
from PIL import Image
import cv2
import numpy
import os
import pandas
from sklearn.metrics import confusion_matrix
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

'''imagePath = './dataset/image/IMAGE_0574.jpg'
imgFile = cv2.imread(imagePath,0)
#print(imgFile.shape)
minVal=numpy.amin(imgFile)
#print('min= ',minVal)
maxVal= numpy.amax(imgFile)
#print('max= ',maxVal)
#mplp.imshow(imgFile)
#mplp.show()'''

#
# load the images
target_size=[64,64]

training_images_dir='./trainingDataset/image'
training_images_paths = [os.path.join(training_images_dir,l) for l in os.listdir(training_images_dir)]
training_images = []
#print(training_images_paths)
for img_path in training_images_paths:
    filename=img_path.split('\\')[-1]
    #print('filename=',filename)
    #target_size=None
    img = image.img_to_array(image.load_img(img_path,target_size=target_size, interpolation='bicubic'))
    #img = image.img_to_array(image.load_img(img_path,target_size=[64,64], interpolation='bicubic'))
    imgBW = img[:,:,0]
    #print(img.shape)
    #print(imgBW.shape)
    '''if (filename=='IMAGE_0000.jpg'):
        mplp.imshow(img/255)
        mplp.show()
        mplp.imshow(imgBW/255)
        mplp.show()'''
    training_images.append(imgBW)
trainingImages=numpy.array(training_images)
print(trainingImages.shape)

test_images_dir='./testDataset/image'
test_images_paths = [os.path.join(test_images_dir,l) for l in os.listdir(test_images_dir)]
test_images = []
for img_path in test_images_paths:
    filename=img_path.split('\\')[-1]
    #print('filename=',filename)
    #target_size=None
    img = image.img_to_array(image.load_img(img_path,target_size=target_size, interpolation='bicubic'))
    #img = image.img_to_array(image.load_img(img_path,target_size=[64,64], interpolation='bicubic'))
    imgBW = img[:,:,0]
    #print(img.shape)
    #print(imgBW.shape)
    '''if (filename=='IMAGE_0000.jpg'):
        mplp.imshow(img/255)
        mplp.show()
        mplp.imshow(imgBW/255)
        mplp.show()'''
    test_images.append(imgBW)
testImages=numpy.array(test_images)
print(testImages.shape)

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term
(3000, 64, 64)
(200, 64, 64)


# Import tumor labels

In [2]:
trainingTumorLabels= pandas.read_csv('./trainingDataset/label.csv')
#print(trainingTumorLabels.shape)
#print(trainingTumorLabels)
trainingTumorLabels=trainingTumorLabels['label']
#print(trainingTumorLabels.shape)
#print(trainingTumorLabels)
'''tumorNumbers = [float(0.0) if label=='no_tumor' else float(1.0) for label in tumorLabels]
for (i,label) in enumerate(tumorLabels):
    if label=='glioma_tumor':
        tumorNumbers[i]=1.0
    else: 
        if label == 'meningioma_tumor':
            tumorNumbers[i]=2.0
        else:
            if label == 'pituitary_tumor':
                tumorNumbers[i]=3.0
            else:
                tumorNumbers[i]=0.0'''
trainingTumorNumbers = [0 if label=='no_tumor' else 1 for label in trainingTumorLabels]
trainingTumorNumbers = numpy.array(trainingTumorNumbers)
print('trainingTumorNumbers is ',trainingTumorNumbers.dtype)
print('trainingTumorNumbers is ',trainingTumorNumbers.shape)
#print(trainingTumorNumbers)
trainingTumorClasses = numpy.array([trainingTumorNumbers, 1-trainingTumorNumbers]).T
print(trainingTumorClasses.shape)
print(trainingTumorClasses)


testTumorLabels= pandas.read_csv('./testDataset/label.csv')
#print(testTumorLabels.shape)
#print(testTumorLabels)
testTumorLabels=testTumorLabels['label']
#print(testTumorLabels.shape)
#print(testTumorLabels)
'''tumorNumbers = [float(0.0) if label=='no_tumor' else float(1.0) for label in tumorLabels]
for (i,label) in enumerate(tumorLabels):
    if label=='glioma_tumor':
        tumorNumbers[i]=1.0
    else: 
        if label == 'meningioma_tumor':
            tumorNumbers[i]=2.0
        else:
            if label == 'pituitary_tumor':
                tumorNumbers[i]=3.0
            else:
                tumorNumbers[i]=0.0'''
testTumorNumbers = [0 if label=='no_tumor' else 1 for label in testTumorLabels]
testTumorNumbers = numpy.array(testTumorNumbers)
print('testTumorNumbers is ',testTumorNumbers.dtype)
print('testTumorNumbers is ',testTumorNumbers.shape)
#print(testTrainingTumorNumbers)
testTumorClasses = numpy.array([testTumorNumbers, 1-testTumorNumbers]).T
print(testTumorClasses.shape)
print(testTumorClasses)


trainingTumorNumbers is  int32
trainingTumorNumbers is  (3000,)
(3000, 2)
[[1 0]
 [0 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
testTumorNumbers is  int32
testTumorNumbers is  (200,)
(200, 2)
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [

# Define an MLP with 1-4 hidden layers

# Input the number of neurons in each layer

In [3]:

def allocate_weights_and_biases(neurons_hidden):
    
    # define number of hidden layers ..
    num_layers = len(neurons_hidden)     

    # inputs placeholders
    X = tf.placeholder("float", [None, 64,64]) # images are 64x64
    Y = tf.placeholder("float", [None, 2])  # 2 output classes
    
    # flatten image features into one vector (i.e. reshape image feature matrix into a vector)
    images_flat = tf.layers.flatten(X)  
    #images_flat = tf.keras.layers.Flatten(X) # this doesn't work
    
    # weights and biases are initialized from a normal distribution with a specified standard devation stddev
    stddev = 0.01
    
    # define placeholders for weights and biases in the graph
    if num_layers==0:
        weights = {
            'out': tf.Variable(tf.random_normal([64*64, 2], stddev=stddev))
        }
        biases = {
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
    if num_layers==1:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
        
    if num_layers==2:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
              
    if num_layers==3:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'hidden_layer3': tf.Variable(tf.random_normal([neurons_hidden[1], neurons_hidden[2]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'bias_layer3': tf.Variable(tf.random_normal([neurons_hidden[2]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
           
    if num_layers==4:
        weights = {
            'hidden_layer1': tf.Variable(tf.random_normal([64*64, neurons_hidden[0]], stddev=stddev)),
            'hidden_layer2': tf.Variable(tf.random_normal([neurons_hidden[0], neurons_hidden[1]], stddev=stddev)),
            'hidden_layer3': tf.Variable(tf.random_normal([neurons_hidden[1], neurons_hidden[2]], stddev=stddev)),
            'hidden_layer4': tf.Variable(tf.random_normal([neurons_hidden[2], neurons_hidden[3]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([neurons_hidden[num_layers-1], 2], stddev=stddev))
        }
        biases = {
            'bias_layer1': tf.Variable(tf.random_normal([neurons_hidden[0]], stddev=stddev)),
            'bias_layer2': tf.Variable(tf.random_normal([neurons_hidden[1]], stddev=stddev)),
            'bias_layer3': tf.Variable(tf.random_normal([neurons_hidden[2]], stddev=stddev)),
            'bias_layer4': tf.Variable(tf.random_normal([neurons_hidden[3]], stddev=stddev)),
            'out': tf.Variable(tf.random_normal([2], stddev=stddev))
        }
    
    return weights, biases, X, Y, images_flat
    

In [4]:
# Create model
def multilayer_perceptron(neurons_hidden):
        
    num_layers = len(neurons_hidden) 
    weights, biases, X, Y, images_flat = allocate_weights_and_biases(neurons_hidden=neurons_hidden)
        
    # Hidden fully connected layer 1
    if (num_layers >=1):
        layer_1 = tf.add(tf.matmul(images_flat, weights['hidden_layer1']), biases['bias_layer1'])
        layer_1 = tf.sigmoid(layer_1)
        layer_final = layer_1

    # Hidden fully connected layer 2
    if (num_layers>=2):
        layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']), biases['bias_layer2'])
        layer_2 = tf.sigmoid(layer_2)
        layer_final = layer_2

    # Hidden fully connected layer 3
    if (num_layers>=3):
        layer_3 = tf.add(tf.matmul(layer_2, weights['hidden_layer3']), biases['bias_layer3'])
        layer_3 = tf.sigmoid(layer_3)
        layer_final = layer_3

    # Hidden fully connected layer 4
    if (num_layers>=4):
        layer_4 = tf.add(tf.matmul(layer_3, weights['hidden_layer4']), biases['bias_layer4'])
        layer_4 = tf.sigmoid(layer_4)
        layer_final = layer_4                     
    
    # Output fully connected layer
    out_layer = tf.matmul(layer_final, weights['out']) + biases['out']

    return out_layer, X, Y



In [5]:
def print_confusion_matrix(predictionClasses, actualClasses,labels):
   #print(testClasses.T)
   #print(predictions_onehot.T)
   print('\nCONFUSION MATRIX')
   print('Actual values in X')
   print('Predicted values in Y')
   confusionMatrix = confusion_matrix(predictionClasses.T[0],actualClasses.T[0])
   print(confusionMatrix,'\n')
 
   inputNumbers = numpy.sum(confusionMatrix, axis=0)
   print(inputNumbers[0],' cases of ',labels[0],': ',round(100*confusionMatrix[0,0]/inputNumbers[0],2),'% chance of correct prediction')
   print(inputNumbers[1],' cases of ',labels[1],': ',round(100*confusionMatrix[1,1]/inputNumbers[1],2),'% chance of correct prediction\n') 

   outputNumbers = numpy.sum(confusionMatrix, axis=1)
   print(outputNumbers[0],' predictions of ',labels[0],' are correct ',round(100*confusionMatrix[0,0]/outputNumbers[0],2),'% of the time')
   print(outputNumbers[1],' predictions of ',labels[1],' are correct ',round(100*confusionMatrix[1,1]/outputNumbers[1],2),'% of the time')
   return

In [6]:
def initialize_and_run_mlp(neurons_hidden,learning_rate, training_epochs):
  # display training accuracy every ..
  display_accuracy_step = 10

  logits, X, Y = multilayer_perceptron(neurons_hidden=neurons_hidden)

  # define loss and optimizer
  loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  # define training graph operation
  train_op = optimizer.minimize(loss_op)

  # graph operation to initialize all variables
  init_op = tf.global_variables_initializer()
    
  with tf.Session() as sess:
    num_sessions = 1
    accuracy_eval = numpy.zeros(num_sessions)
    
    for session in range(num_sessions):
        # run graph weights/biases initialization op
        sess.run(init_op)
        # begin training loop ..
        for epoch in range(training_epochs):
            # complete code below
            # run optimization operation (backprop) and cost operation (to get loss value)
            #_, cost = sess.run([train_op, loss_op], feed_dict={X: training_images,
                                                               #Y: trainingClasses})
            _, cost = sess.run([train_op, loss_op], feed_dict={X: trainingImages,
                                                               Y: trainingTumorClasses})

            # Display logs per epoch step
            #print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(cost))
                
            if epoch % display_accuracy_step == 0:
                pred = tf.nn.softmax(logits)  # Apply softmax to logits
                correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

                # calculate training accuracy
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                #print("Accuracy: {:.3f}".format(accuracy.eval({X: training_images, Y: trainingClasses})))

        print('\nSession ',session+1," Optimization Finished!")

        # -- Define and run test operation -- #
        
        # apply softmax to output logits
        pred = tf.nn.softmax(logits)
        
        #  derive inffered calasses as the class with the top value in the output density function
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        
        # calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
        # run test accuracy operation ..
        accuracy_eval[session] = accuracy.eval({X: testImages, Y: testTumorClasses})
        print('Session ',session+1," Test Accuracy:", round(100*accuracy_eval[session],2),'%')
        
        predictions = pred.eval(feed_dict = {X:testImages})
        #print(predictions.T)
        predictions_onehot = predictions.copy()
        for i, prediction in enumerate(predictions):
            #print(i,'   ',prediction)
            if prediction[0] > prediction[1]:
                predictions_onehot[i] = [1,0]
            else:
                predictions_onehot[i] = [0,1]
        #print(predictions_onehot.T)
        print_confusion_matrix(predictions_onehot,testTumorClasses, labels=['no tumor','a tumor'])
        
    print('\nMean Test Accuracy = ',round(100*numpy.mean(accuracy_eval),2),'%\n\n')
    return(predictions_onehot, testTumorClasses)
      



In [9]:
 #dummy command to get rid of deprecation warning.
predictions_onehot, testTumorClasses = initialize_and_run_mlp([1], learning_rate=0.01, training_epochs=1)



Session  1  Optimization Finished!
Session  1  Test Accuracy: 81.5 %

CONFUSION MATRIX
Actual values in X
Predicted values in Y
[[  0   0]
 [ 37 163]] 

37  cases of  no tumor :  0.0 % chance of correct prediction
163  cases of  a tumor :  100.0 % chance of correct prediction

0  predictions of  no tumor  are correct  nan % of the time
200  predictions of  a tumor  are correct  81.5 % of the time

Mean Test Accuracy =  81.5 %




C:\Users\randy\anaconda3\envs\Python3pt6\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in longlong_scalars
  from ipykernel import kernelapp as app


In [10]:
learning_rate = 1e-5
training_epochs = 500
neurons_hidden = [2048,2048,2048,2048]
num_layers = len(neurons_hidden) # default
num_layers = 1  # override
for num_layers in range(4,0,-1):
    neurons_hidden = neurons_hidden[:num_layers]
    print(num_layers,' hidden layers')
    print(neurons_hidden,' neurons')
    predictions_onehot, testTumorClasses = initialize_and_run_mlp(neurons_hidden=neurons_hidden, learning_rate=learning_rate, training_epochs=training_epochs)



4  hidden layers
[2048, 2048, 2048, 2048]  neurons

Session  1  Optimization Finished!
Session  1  Test Accuracy: 92.5 %

CONFUSION MATRIX
Actual values in X
Predicted values in Y
[[ 24   2]
 [ 13 161]] 

37  cases of  no tumor :  64.86 % chance of correct prediction
163  cases of  a tumor :  98.77 % chance of correct prediction

26  predictions of  no tumor  are correct  92.31 % of the time
174  predictions of  a tumor  are correct  92.53 % of the time

Mean Test Accuracy =  92.5 %


3  hidden layers
[2048, 2048, 2048]  neurons

Session  1  Optimization Finished!
Session  1  Test Accuracy: 92.5 %

CONFUSION MATRIX
Actual values in X
Predicted values in Y
[[ 23   1]
 [ 14 162]] 

37  cases of  no tumor :  62.16 % chance of correct prediction
163  cases of  a tumor :  99.39 % chance of correct prediction

24  predictions of  no tumor  are correct  95.83 % of the time
176  predictions of  a tumor  are correct  92.05 % of the time

Mean Test Accuracy =  92.5 %


2  hidden layers
[2048, 20

# MLP with SGD (instead of Adam)

In [56]:
def initialize_and_run_mlp_with_sgd(neurons_hidden,learning_rate, training_epochs):
  # display training accuracy every ..
  display_accuracy_step = 10

  logits, X, Y = multilayer_perceptron(neurons_hidden=neurons_hidden)

  # define loss and optimizer
  loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
  #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  #var = tf.Variable(1.0)
  #loss = lambda: (var ** 2)/2.0         # d(loss)/d(var1) = var1
  #step_count = opt.minimize(loss, [var]).numpy()
  # Step is `- learning_rate * grad`
  #var.numpy()

  # define training graph operation
  #train_op = optimizer.minimoze(loss_op)
  train_op = optimizer.minimize(loss_op, var_list = None)
  #train_op = optimizer.minimize(loss_op, var_list = logits)
  #train_op = optimizer.minimize(loss_op, var_list = weights)
  
  
  # graph operation to initialize all variables
  init_op = tf.global_variables_initializer()
    
  with tf.Session() as sess:
    num_sessions = 1
    accuracy_eval = numpy.zeros(num_sessions)
    
    for session in range(num_sessions):
        # run graph weights/biases initialization op
        sess.run(init_op)
        # begin training loop ..
        for epoch in range(training_epochs):
            # complete code below
            # run optimization operation (backprop) and cost operation (to get loss value)
            _, cost = sess.run([train_op, loss_op], feed_dict={X: trainingImages,
                                                               Y: trainingTumorClasses})

            # Display logs per epoch step
            #print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(cost))
                
            if epoch % display_accuracy_step == 0:
                pred = tf.nn.softmax(logits)  # Apply softmax to logits
                correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

                # calculate training accuracy
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                #print("Accuracy: {:.3f}".format(accuracy.eval({X: training_images, Y: trainingClasses})))

        print('\nSession ',session+1," Optimization Finished!")

        # -- Define and run test operation -- #
        
        # apply softmax to output logits
        pred = tf.nn.softmax(logits)
        
        #  derive inffered calasses as the class with the top value in the output density function
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        
        # calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
        # run test accuracy operation ..
        accuracy_eval[session] = accuracy.eval({X: testImages, Y: testTumorClasses})
        print('Session ',session+1," Test Accuracy:", round(100*accuracy_eval[session],2),'%')
        
        predictions = pred.eval(feed_dict = {X:testImages})
        #print(predictions.T)
        predictions_onehot = predictions.copy()
        for i, prediction in enumerate(predictions):
            #print(i,'   ',prediction)
            if prediction[0] > prediction[1]:
                predictions_onehot[i] = [1,0]
            else:
                predictions_onehot[i] = [0,1]
        #print(predictions_onehot.T)
        print_confusion_matrix(predictions_onehot,testTumorClasses, labels=['no tumor','a tumor'])
        
    print('\nMean Test Accuracy = ',round(100*numpy.mean(accuracy_eval),2),'%\n\n')
    return(predictions_onehot, testTumorClasses)

In [57]:
learning_rate = 1e-5
training_epochs = 500
neurons_hidden = [2048,2048,2048,2048]
num_layers = len(neurons_hidden) # default
num_layers = 1  # override

neurons_hidden = neurons_hidden[:num_layers]    
print(num_layers,' hidden layers')
print(neurons_hidden,' neurons')
predictions_onehot, testTumorClasses = initialize_and_run_mlp_with_sgd(neurons_hidden=neurons_hidden, learning_rate=learning_rate, training_epochs=training_epochs)



1  hidden layers
[2048]  neurons


ValueError: Passed in object of type <class 'NoneType'>, not tf.Tensor

# MPL with dropconnect

In [43]:
# Create model
def multilayer_perceptron_with_dropconnect(neurons_hidden):
        
    num_layers = len(neurons_hidden) 
    weights, biases, X, Y, images_flat = allocate_weights_and_biases(neurons_hidden=neurons_hidden)
    
    keep_prob = tf.placeholder(dtype=tf.float32)
    keep_prob = 0.5
    rate=1 - keep_prob
        
    # Hidden fully connected layer 1
    if (num_layers >=1):
        dropConnect = tf.nn.dropout(weights['hidden_layer1'], rate) * rate
        layer_1 = tf.add(tf.matmul(images_flat, dropConnect), biases['bias_layer1'])
        layer_1 = tf.sigmoid(layer_1)
        layer_final = layer_1

    # Hidden fully connected layer 2
    if (num_layers>=2):
        dropConnect = tf.nn.dropout(weights['hidden_layer2'], rate) * rate
        layer_2 = tf.add(tf.matmul(layer_1, dropConnect), biases['bias_layer2'])
        layer_2 = tf.sigmoid(layer_2)
        layer_final = layer_2

    # Hidden fully connected layer 3
    if (num_layers>=3):
        dropConnect = tf.nn.dropout(weights['hidden_layer3'], rate) * rate
        layer_3 = tf.add(tf.matmul(layer_2, dropConnect), biases['bias_layer3'])
        layer_3 = tf.sigmoid(layer_3)
        layer_final = layer_3

    # Hidden fully connected layer 4
    if (num_layers>=4):
        dropConnect = tf.nn.dropout(weights['hidden_layer4'], rate) * rate
        layer_4 = tf.add(tf.matmul(layer_3, dropConnect), biases['bias_layer4'])
        layer_4 = tf.sigmoid(layer_4)
        layer_final = layer_4                     
    
    # Output fully connected layer
    dropConnect = tf.nn.dropout(weights['out'], rate) * rate
    out_layer = tf.matmul(layer_final, dropConnect) + biases['out']

    return out_layer, X, Y



In [44]:
def initialize_and_run_mlp_with_dropconnect(neurons_hidden,learning_rate, training_epochs):
  # display training accuracy every ..
  display_accuracy_step = 10

  logits, X, Y = multilayer_perceptron_with_dropconnect(neurons_hidden=neurons_hidden)

  # define loss and optimizer
  loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

  # define training graph operation
  train_op = optimizer.minimize(loss_op)

  # graph operation to initialize all variables
  init_op = tf.global_variables_initializer()
    
  with tf.Session() as sess:
    num_sessions = 1
    accuracy_eval = numpy.zeros(num_sessions)
    
    for session in range(num_sessions):
        # run graph weights/biases initialization op
        sess.run(init_op)
        # begin training loop ..
        for epoch in range(training_epochs):
            # complete code below
            # run optimization operation (backprop) and cost operation (to get loss value)
            #_, cost = sess.run([train_op, loss_op], feed_dict={X: training_images,
                                                               #Y: trainingClasses})
            _, cost = sess.run([train_op, loss_op], feed_dict={X: trainingImages,
                                                               Y: trainingTumorClasses})

            # Display logs per epoch step
            #print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(cost))
                
            if epoch % display_accuracy_step == 0:
                pred = tf.nn.softmax(logits)  # Apply softmax to logits
                correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

                # calculate training accuracy
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                #print("Accuracy: {:.3f}".format(accuracy.eval({X: training_images, Y: trainingClasses})))

        print('\nSession ',session+1," Optimization Finished!")

        # -- Define and run test operation -- #
        
        # apply softmax to output logits
        pred = tf.nn.softmax(logits)
        
        #  derive inffered calasses as the class with the top value in the output density function
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        
        # calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
        # run test accuracy operation ..
        accuracy_eval[session] = accuracy.eval({X: testImages, Y: testTumorClasses})
        print('Session ',session+1," Test Accuracy:", round(100*accuracy_eval[session],2),'%')
        
        predictions = pred.eval(feed_dict = {X:testImages})
        #print(predictions.T)
        predictions_onehot = predictions.copy()
        for i, prediction in enumerate(predictions):
            #print(i,'   ',prediction)
            if prediction[0] > prediction[1]:
                predictions_onehot[i] = [1,0]
            else:
                predictions_onehot[i] = [0,1]
        #print(predictions_onehot.T)
        print_confusion_matrix(predictions_onehot,testTumorClasses, labels=['no tumor','a tumor'])
        
    print('\nMean Test Accuracy = ',round(100*numpy.mean(accuracy_eval),2),'%\n\n')
    return(predictions_onehot, testTumorClasses)

In [45]:
learning_rate = 1e-5
training_epochs = 500
neurons_hidden = [2048,2048,2048,2048]
num_layers = len(neurons_hidden) # default
num_layers = 1  # override

neurons_hidden = neurons_hidden[:num_layers]    
print(num_layers,' hidden layers')
print(neurons_hidden,' neurons')
predictions_onehot, testTumorClasses = initialize_and_run_mlp_with_dropconnect(neurons_hidden=neurons_hidden, learning_rate=learning_rate, training_epochs=training_epochs)



1  hidden layers
[2048]  neurons


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_49' with dtype float
	 [[node Placeholder_49 (defined at <ipython-input-43-28b03774b027>:7) ]]
	 [[Mean_381/_13]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_49' with dtype float
	 [[node Placeholder_49 (defined at <ipython-input-43-28b03774b027>:7) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder_49':
  File "C:\Users\randy\anaconda3\envs\Python3pt6\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\randy\anaconda3\envs\Python3pt6\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\randy\anaconda3\envs\Python3pt6\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\randy\anaconda3\envs\Python3pt6\lib\site-packages\traitlets\config\application.py", line 66